In [8]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import iris
import iris.quickplot as qplt
import iris.plot as iplt
from iris.experimental.regrid import regrid_weighted_curvilinear_to_rectilinear
import json
import cftime
from itertools import product
from cftime import DatetimeNoLeap
#import iris_utils
#import Gridding
#from nc_processing import *
#from JASMIN_utils import *
#from analysis import * 
#from plotting import *
import esmvalcore.preprocessor
import xesmf as xe
import warnings
%matplotlib inline
import tqdm
import seaborn as sns
#sns.set()

from datatree import DataTree
from xmip.preprocessing import rename_cmip6


def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique))

In [2]:
experiment = "ssp370"
dirs = []
var_path = "Amon/tas"

for x in glob.glob('/badc/cmip6/data/CMIP6/ScenarioMIP/*/*/{e}/*/{v}/*/latest/'.format(e=experiment, v=var_path)):
    dirs.append(x)


In [3]:
#cut down:
dirs = dirs[0:5]

In [4]:
names = []
for folder in dirs:
    name = str(folder.split('/')[7] + "/" + folder.split('/')[8] + "/" + folder.split('/')[9])
    names.append(name)

cmip_stores = dict(zip(names, dirs))  

datasets = {
    name: rename_cmip6(xr.open_mfdataset(path + "*.nc", use_cftime=True))#.load()
    for name, path in cmip_stores.items()
}

In [5]:
dt = DataTree.from_dict(datasets)

print(f"Size of data in tree = {dt.nbytes / 1e9 :.2f} GB")
dt

#~3GB per 10 models, ~60GB for all available

Size of data in tree = 1.48 GB


DataTree('None', parent=None)
├── DataTree('TaiESM1')
│   └── DataTree('ssp370')
│       └── DataTree('r1i1p1f1')
│               Dimensions:      (time: 1032, bnds: 2, y: 192, x: 288)
│               Coordinates:
│                 * time         (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
│                 * y            (y) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
│                 * x            (x) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
│               Dimensions without coordinates: bnds
│               Data variables:
│                   time_bounds  (time, bnds) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
│                   lat_bounds   (y, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
│                   lon_bounds   (x, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
│                   height       float64 ...
│                   tas          (time, y, x) float32 dask.array<chunksize=(1032, 192, 288), meta=np.ndarray>
│               Attributes: (12/50)
│                   Conventions:               CF-1.7 CMIP-6.2
│                   activity_id:               ScenarioMIP AerChemMIP
│                   branch_method:             Hybrid-restart from year 2015-01-01 of historical
│                   branch_time:               0.0
│                   branch_time_in_child:      0.0
│                   branch_time_in_parent:     60225.0
│                   ...                        ...
│                   title:                     TaiESM1 output prepared for CMIP6
│                   variable_id:               tas
│                   variant_label:             r1i1p1f1
│                   license:                   CMIP6 model data produced by NCC is licensed u...
│                   cmor_version:              3.5.0
│                   tracking_id:               hdl:21.14100/bb8d43fb-449a-4c00-a408-d72a152464a9
└── DataTree('AWI-CM-1-1-MR')
    └── DataTree('ssp370')
        ├── DataTree('r1i1p1f1')
        │       Dimensions:      (time: 1032, bnds: 2, y: 192, x: 384)
        │       Coordinates:
        │         * time         (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
        │         * y            (y) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
        │         * x            (x) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
        │       Dimensions without coordinates: bnds
        │       Data variables:
        │           time_bounds  (time, bnds) object dask.array<chunksize=(12, 2), meta=np.ndarray>
        │           lat_bounds   (time, y, bnds) float64 dask.array<chunksize=(12, 192, 2), meta=np.ndarray>
        │           lon_bounds   (time, x, bnds) float64 dask.array<chunksize=(12, 384, 2), meta=np.ndarray>
        │           height       float64 2.0
        │           tas          (time, y, x) float32 dask.array<chunksize=(12, 192, 384), meta=np.ndarray>
        │       Attributes: (12/44)
        │           Conventions:            CF-1.7 CMIP-6.2
        │           activity_id:            ScenarioMIP AerChemMIP
        │           branch_method:          standard
        │           branch_time_in_child:   0.0
        │           branch_time_in_parent:  60265.0
        │           creation_date:          2019-09-27T14:01:00Z
        │           ...                     ...
        │           title:                  AWI-CM-1-1-MR output prepared for CMIP6
        │           variable_id:            tas
        │           variant_label:          r1i1p1f1
        │           license:                CMIP6 model data produced by Alfred Wegener Insti...
        │           cmor_version:           3.4.0
        │           tracking_id:            hdl:21.14100/c572c445-daf1-45e2-bd6f-f6033e97ac1a
        ├── DataTree('r2i1p1f1')
        │       Dimensions:      (time: 1032, bnds: 2, y: 192, x: 384)
        │       Coordinates:
        │         * time         (time) ob

In [6]:
datasets

{'TaiESM1/ssp370/r1i1p1f1': <xarray.Dataset>
 Dimensions:      (time: 1032, bnds: 2, y: 192, x: 288)
 Coordinates:
   * time         (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
   * y            (y) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
   * x            (x) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
 Dimensions without coordinates: bnds
 Data variables:
     time_bounds  (time, bnds) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
     lat_bounds   (y, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
     lon_bounds   (x, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
     height       float64 ...
     tas          (time, y, x) float32 dask.array<chunksize=(1032, 192, 288), meta=np.ndarray>
 Attributes: (12/50)
     Conventions:               CF-1.7 CMIP-6.2
     activity_id:               ScenarioMIP AerChemMIP
     branch_method:             Hybrid-restart from year 2015-01-01 of historical
     branc

In [12]:
dset_dict = datasets
dataset_list = list(dset_dict.keys())

matches = []

for i in dataset_list:
    # print(i)
    
    if '370' in i:
        
        model_config = '.'.join(i.split('.')[1:3])
        
        print(model_config)
        
        x = []
        
        for j in dataset_list:
            
            if model_config in j:
                
                x.append(j)
              
        matches.append(sorted(x))


output = {}

for k in tqdm.tqdm(matches[:-1]):
    
    print(k)
    
    # try:
        
    a = []
    w = []
    y = []

    for i in k:

        data = dset_dict[i]

        winter_mask = data.time.dt.month.isin([12,1,2])
        jan_mask = data.time.dt.month.isin([1]) 
        name = " ".join(i.split('.')[1:3])

        at_data = data['tas']#.isel(member_id=0)

        at_data = at_data[winter_mask]

        years = data.time.dt.year[jan_mask].compute()

        # month_length = data.time.dt.days_in_month
        # weights = ( month_length.groupby("time.season") / month_length.groupby("time.season").mean())
        world_annual = (at_data).groupby("time.year").mean(dim="time")

        arctic_annual = world_annual.sel(y=slice(-90,66))

        world_w = world_annual.weighted(weights=np.cos(np.deg2rad(world_annual.y)))
        arctic_w = arctic_annual.weighted(weights=np.cos(np.deg2rad(arctic_annual.y)))

        w += list(world_w.mean(("x","y")).compute())
        a += list(arctic_w.mean(("x","y")).compute())
        y += list(years)

    output[name] = {'arctic':a,
                    'world':w,
                    'year':y,
                   }
        
#         break
        
#     except:
#         pass

df.set_index('year',inplace=True)
for name, d in output.items():
    
    try:

        df = pd.DataFrame(d)
        
        df.set_index('year',inplace=True)
        
        df.sort_index(inplace=True)
        
        plt.plot(df.index,np.convolve(df['arctic']-np.nanmean(df['arctic'][df.index<1900]),np.ones(9)/9,mode='same'))
        plt.plot(df.index,np.convolve(df['world']-np.nanmean(df['world'][df.index<1900]),np.ones(9)/9,mode='same'))
        
        plt.axhline(1.5)
        
        plt.xlim(1850,2050)
        
        # plt.show()
        
        # break
    
    except:
        print(name)

  0%|          | 0/4 [00:00<?, ?it/s]

['AWI-CM-1-1-MR/ssp370/r1i1p1f1', 'AWI-CM-1-1-MR/ssp370/r2i1p1f1', 'AWI-CM-1-1-MR/ssp370/r3i1p1f1', 'AWI-CM-1-1-MR/ssp370/r4i1p1f1', 'TaiESM1/ssp370/r1i1p1f1']


 25%|██▌       | 1/4 [01:08<03:25, 68.43s/it]

['AWI-CM-1-1-MR/ssp370/r1i1p1f1', 'AWI-CM-1-1-MR/ssp370/r2i1p1f1', 'AWI-CM-1-1-MR/ssp370/r3i1p1f1', 'AWI-CM-1-1-MR/ssp370/r4i1p1f1', 'TaiESM1/ssp370/r1i1p1f1']


 50%|█████     | 2/4 [01:58<01:55, 57.65s/it]

['AWI-CM-1-1-MR/ssp370/r1i1p1f1', 'AWI-CM-1-1-MR/ssp370/r2i1p1f1', 'AWI-CM-1-1-MR/ssp370/r3i1p1f1', 'AWI-CM-1-1-MR/ssp370/r4i1p1f1', 'TaiESM1/ssp370/r1i1p1f1']


 75%|███████▌  | 3/4 [02:44<00:52, 52.13s/it]

['AWI-CM-1-1-MR/ssp370/r1i1p1f1', 'AWI-CM-1-1-MR/ssp370/r2i1p1f1', 'AWI-CM-1-1-MR/ssp370/r3i1p1f1', 'AWI-CM-1-1-MR/ssp370/r4i1p1f1', 'TaiESM1/ssp370/r1i1p1f1']


100%|██████████| 4/4 [03:28<00:00, 52.12s/it]


NameError: name 'df' is not defined